In [1]:
import pandas as pd
import os 
import geopandas as gpd
import numpy as np
import geemap
import ee
from datetime import datetime

#### Authenticating my Google earth Engine Cloud Project

Free to set up on GEE Cloud. Will be prompted to authenticate in browser and enter a code in IDE for first-time use.

In [2]:
ee.Authenticate()
# rename the project your own GEE cloud project
ee.Initialize(project = 'biltong-465000')

#### Load in Meat Naturally Polygons

In [7]:
boundaries = gpd.read_file("../Data/250520_GRASS_Active Boundaries.gpkg")
sub_divs = gpd.read_file("../Data/250601_GRASS_Association Sub-Divisions.gpkg")

# convert to EE feature collections
boundaries_fc = geemap.geopandas_to_ee(boundaries)
sub_divs_fc = geemap.geopandas_to_ee(sub_divs)

#### Helper Functions to Process Landsat Imagery

In [ ]:
def mask_landsat_sr(image):
    
    """ Masks Landsat imagery to remove clouds, shadows, & snow before analysis"""

    # built-in Landsat QA_PIXEL band helps us ID reflective surfaces
    qa = image.select("QA_PIXEL")
    mask = (
        qa.bitwiseAnd(1 << 3).eq(0)  # shadow
        .And(qa.bitwiseAnd(1 << 4).eq(0))  # snow
        .And(qa.bitwiseAnd(1 << 5).eq(0))  # cloud
        .And(qa.bitwiseAnd(1 << 7).eq(0))  # cirrus
    )
    return image.updateMask(mask)

def get_collection(year, fc, max_cloud_cover = 20):
    
    """ 
    Returns the appropriate Landsat collection for a given year and feature collection
    Calculates NDVI and stores it in a new NDVI band
    """

    start = ee.Date.fromYMD(year, 1, 1)
    end   = ee.Date.fromYMD(year, 12, 31)
    if year < 2014:
        col, red, nir = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2"), "SR_B3", "SR_B4"
    else:
        col, red, nir = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2"), "SR_B4", "SR_B5"
    return (
        col
         .filterDate(start, end)
         .filterBounds(fc)
         .filter(ee.Filter.lt("CLOUD_COVER", max_cloud_cover))
         .map(mask_landsat_sr)
         .map(lambda img: img
              .select([nir, red])
              .multiply(0.0000275) #Landsat SR scaling factor
              .normalizedDifference([nir, red])
              .rename("NDVI"))
    )

def ee_fc_to_gdf(fc):

    """ Converts an Earth Engine FeatureCollection to a GeoDataFrame """ 

    # this seems like it'd be built-in but I couldn't find it in API

    gj = fc.getInfo()
    return gpd.GeoDataFrame.from_features(gj['features'])
                                          
def process_fc(fc, region_name):

    """ Processes a feature collection of imagery to extract NDVI statistics for each year """

    dfs = []

    for year in range(2000, 2024):
        print(f"  • {region_name}: Processing {year}")
        ndvi_mean = get_collection(year, fc).mean()

        stats = ndvi_mean.reduceRegions(
            collection = fc,
            reducer    = ee.Reducer.mean(),
            scale      = 30,
            tileScale  = 4
        ).map(lambda f: f.set("year", year))

        df = ee_fc_to_gdf(stats)
        dfs.append(df)

    return pd.concat(dfs, ignore_index=True)

Processing NDVI for Boundary & Subdivision Boundaries

In [ ]:
print("Starting boundary-wide NDVI…")
ndvi_boundaries = process_fc(boundaries_fc, "boundaries")
print("Starting subdivision NDVI…")
ndvi_subdivisions = process_fc(sub_divs_fc, "subdivisions")

ndvi_boundaries = ndvi_boundaries.set_crs(epsg=4326)
ndvi_subdivisions = ndvi_subdivisions.set_crs(epsg=4326)

Starting country‐wide NDVI…
  • country: Processing 2000
  • country: Processing 2001
  • country: Processing 2002
  • country: Processing 2003
  • country: Processing 2004
  • country: Processing 2005
  • country: Processing 2006
  • country: Processing 2007
  • country: Processing 2008
  • country: Processing 2009
  • country: Processing 2010
  • country: Processing 2011
  • country: Processing 2012
  • country: Processing 2013
  • country: Processing 2014
  • country: Processing 2015
  • country: Processing 2016
  • country: Processing 2017
  • country: Processing 2018
  • country: Processing 2019
  • country: Processing 2020
  • country: Processing 2021
  • country: Processing 2022
  • country: Processing 2023
Starting subdivision NDVI…
  • subdivision: Processing 2000
  • subdivision: Processing 2001
  • subdivision: Processing 2002
  • subdivision: Processing 2003
  • subdivision: Processing 2004
  • subdivision: Processing 2005
  • subdivision: Processing 2006
  • subdivision: P

C:\Users\zjsch\AppData\Roaming\Python\Python311\site-packages\pyogrio\geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(
C:\Users\zjsch\AppData\Roaming\Python\Python311\site-packages\pyogrio\geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(


#### Pivoting GDFs 

Process above adds a new record in GDFs for each year, this moves avg NDVI info to be stored in columns. Should redo code above later to avoid this.

In [ ]:
wide_boundaries = ndvi_boundaries.pivot_table(
    index=['Active_Carbon', 'Active_RSA', 'Area_ha', 'Association_ID', 'Association_Official',	'Elig_ha', 'GrazingUnit_ID', 'GrazingUnit_Name', 'Implementing_Partner', 'Landscape_Name', 'Province', 'Start_Year', 'geometry'],
    columns='year',
    values='mean'
).reset_index()

new_cols = {}
for col in wide_boundaries.columns:
    try:
        year = int(col)
        new_cols[col] = f"ndvi_{year}"
    except (TypeError, ValueError):
        # non-year columns stay unchang
        pass

wide_boundaries = wide_boundaries.rename(columns = new_cols)

gdf_boundaries = gpd.GeoDataFrame(
    wide_boundaries,
    geometry='geometry',
    crs = ndvi_boundaries.crs
)


wide_subdivisions = ndvi_subdivisions.pivot_table(
    index=[	'Area_ha', 'GrazingUnit_ID', 'SubDiv_ID', 'SubDiv_Seq', 'geometry'],
    columns='year',
    values='mean'
).reset_index()

new_cols = {}
for col in wide_subdivisions.columns:
    try:
        year = int(col)
        new_cols[col] = f"ndvi_{year}"
    except (TypeError, ValueError):
        # non-year columns stay unchanged
        pass

wide_subdivisions = wide_subdivisions.rename(columns = new_cols)

gdf_subdivisions = gpd.GeoDataFrame(
    wide_subdivisions,
    geometry='geometry',
    crs = ndvi_subdivisions.crs
)

gdf_boundaries.to_file("GRASS_Boundaries_NDVI.gpkg", driver="GPKG", layer="ndvi")
gdf_subdivisions.to_file("GRASS_Subdvisions_NDVI.gpkg", driver="GPKG", layer="ndvi")